In [1]:
import json
from tqdm import tqdm
from glob import glob
from torch.utils.data import DataLoader, Dataset
from streaming import LocalDataset
from datasets import Audio

In [2]:
split_by = 100_000

In [3]:
!rm -rf partition-audio-vad
!mkdir partition-audio-vad

In [4]:
class Train(Dataset):
    def __init__(self, folder):
        if folder.endswith('.json'):
            with open(folder) as fopen:
                self.data = json.load(fopen)
        else:
            self.data = LocalDataset(folder)
        self.audio = Audio(sampling_rate=16000)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, item):
        try:
            return self.data[item]
        except Exception as e:
            print(e)

            return None

In [5]:
data = Train('mosaic-whisper-filtered-fasttext')

In [6]:
data[5000000]

{'audio_filename': 'output-audio/1-10790-22.mp3',
 'text': '<|startoftranscript|><|ms|><|transcribe|><|0.02|> tidak bercadang untuk selaraskan<|2.42|><|2.42|> cuti kepada<|4.36|><|4.36|> seluruh negeri di negara kita ini<|6.18|><|6.18|> pada hari Jumat dan Sabtu<|7.54|><|7.54|><|11.66|><|11.66|> Datuk yang dipertua<|14.20|><|14.20|> terima kasih kepada<|15.04|><|15.04|> yang berhormat Sen. Datuk Razali<|18.24|><|18.24|> sebenarnya soalan ini out of contact<|19.96|><|19.96|> daripada apa yang kita<|22.22|><|22.22|> soalan dibangkitkan peringkat awal tadi<|23.98|><|23.98|> jadi bila kita ada cadangan dibincangkan<|26.20|><|26.20|> baru dibincang oleh kerajaan tapi benda ini tak ada langsung<|28.48|><|28.48|> ini nak bincang apa?<|29.78|><|endoftext|>'}

In [7]:
index = 0
count = 0
a = open(f'partition-audio-vad/audio_file-{index}.jsonl', 'w')

for l in tqdm(range(len(data))):
    a.write(f'{json.dumps(data[l])}\n')
    a.flush()
    count += 1
    if count >= split_by:
        a.close()
        index += 1
        count = 0
        a = open(f'partition-audio-vad/audio_file-{index}.jsonl', 'w')

if not a.closed:
    a.close()


100%|██████████| 7664890/7664890 [05:53<00:00, 21684.41it/s]


In [8]:
with open(f'partition-audio-vad/audio_file-0.jsonl') as f:
    for x in f:
        print(json.loads(x))
        break

{'audio_filename': 'split-nusantara/0-0-0.mp3', 'text': '<|startoftranscript|><|ms|><|transcribe|><|0.52|> Bab 3<|1.78|><|1.78|> Orang Turk memperjuangkan hak mereka.<|6.22|><|7.36|> Orang ramai lalu-lalang melintasi arca Hassan Tahsin<|10.86|><|10.86|> yang berdiri di Medan Konak,<|13.38|><|13.72|> iaitu bangunan kerajaan,<|14.82|><|15.22|> yang terletak berhampiran dermaga Izmir<|17.44|><|17.44|> yang tersergam indah<|19.36|><|19.36|> dan baru siap dibina semula.<|21.40|><|22.38|> Hassan Tahsin ialah nama samaran seorang wartawan<|25.10|><|25.10|> yang telah menembak mati pembawa panji pasukan tentera<|28.36|><|endoftext|>'}
